<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Analysis/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls drive/MyDrive/CS/Thesis/

 GPT2	        LDA_NRC.png				        rc_testdata
 LDA_all.html   LDA_Trouw.html				        Testset_2021
 LDA_all.png    LDA_Trouw.png				        volkskrant-nl
 LDA_NRC.html  'Political Influence of Dutch Newspapers.pptx'


# Vanilla

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="GroNLP/gpt2-small-dutch")

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/108 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [ ]:
pipe("Mark Rutte", max_length=30, num_return_sequences=5)

[{'generated_text': "Mark Rutte: 'We zijn nog niet klaar om een deal te maken'\nDe Europese Unie is akkoord gegaan met het principe-akkoord tussen Oekraïne en"},
 {'generated_text': 'Mark Rutte (VVD-leider) zei: "Er is een grote kloof tussen VVD en D66 op de arbeidsmarkt. Die verschillen niet."\nDe'},
 {'generated_text': "Mark Rutte (VVD), VVD-Tweede Kamerlid en minister van Buitenlandse Zaken: 'Ik denk dat we het niet meer kunnen veroorloven, maar ik kan me"},
 {'generated_text': 'Mark Rutte, die als premier van het Europees Parlement een akkoord had bereikt met Turkije op de toetreding tot de EU, zei gisteren in zijn wekelijkse persconferentie dat'},
 {'generated_text': 'Mark Rutte: "Ik wil dat er een einde komt aan het geweld tussen moslims en christenen die de afgelopen jaren zo\'n verschrikkelijke tragedie teweegbrengt.'}]

In [ ]:
pipe("Vluchtelingen hebben", max_length=30, num_return_sequences=5)

[{'generated_text': "Vluchtelingen hebben het leven zuur gemaakt. Maar de meesten krijgen er wel degelijk van.'\n'Ik kan je verzekeren dat mijn vader is gestorven,'"},
 {'generated_text': 'Vluchtelingen hebben in totaal 1,6 miljard dollar (8 miljoen euro) uitgetrokken om het land uit te breiden. Het ministerie van Financiën zegt dat'},
 {'generated_text': "Vluchtelingen hebben geen idee dat ze hier zijn.\n'We kunnen niet in de lucht komen,' zei hij met een blik naar buiten, '"},
 {'generated_text': "Vluchtelingen hebben het gevoel dat ze niet langer veilig zijn. Een paar jaar geleden zei een van hen: 'Je hoeft je geen zorgen te maken"},
 {'generated_text': "Vluchtelingen hebben geen recht op asiel of verblijf in het buitenland. Dat is de conclusie van Vluchtelingenwerk Nederland.'\nDe Immigratie- en Natur"}]

In [ ]:
pipe("Drugs zijn", max_length=30, num_return_sequences=5)

[{'generated_text': 'Drugs zijn al een belangrijke inkomstenbron van het Nederlandse leger, zo bleek onlangs.\nHet is niet duidelijk wat er precies in de drugsbestrijding'},
 {'generated_text': "Drugs zijn ook nog niet altijd goed voor je lichaam.'\n'Ja. Dat is precies het probleem met de meeste mensen, en ik zie dat"},
 {'generated_text': "Drugs zijn nog altijd een probleem in de stad.\n'Het is maar één ding', zei ze tegen hem,'maar je weet wel dat"},
 {'generated_text': "Drugs zijn ook wel een van de manieren waarop je er op kunt letten.'\n'Het is goed dat hij het niet wil,' zei ze."},
 {'generated_text': "Drugs zijn een manier van om je te beschermen tegen drugs.'\n 'Ik weet niet of ik er iets mee kan doen. Ik denk dat we"}]

In [ ]:
pipe("Het CDA", max_length=30, num_return_sequences=5)

[{'generated_text': 'Het CDA\'er is een van de grootste voorstanders van het nieuwe kinderpardon, zegt ze. "Ik wil dat Nederland niet alleen asielzoekers uit andere landen'},
 {'generated_text': 'Het CDA-fractie in het Eerste Kamer is van plan een nieuwe coalitie te vormen tussen VVD en D66, die de PvdA niet wil.\nDe'},
 {'generated_text': "Het CDA-fractievoorzitter in de Tweede Kamer heeft gisteren gezegd dat hij,,zeer tevreden'' is over het onderzoek. De PvdA en VVD gaan op zoek naar"},
 {'generated_text': "Het CDA-leider Buma vindt dat Nederland de beste oplossing voor het probleem is. Maar hij wil niet zeggen: 'Als je er iets van begrijpt,"},
 {'generated_text': 'Het CDA-minister Van Boxtel zei in een tv-interview eerder dit jaar aan de orde:,,Ik denk dat er nog veel mensen zijn die zich'}]

# Finetuning

In [12]:
import pandas as pd
import re

#df = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/NRC_2021.json')
df = pd.read_json('drive/MyDrive/CS/Thesis/Testset_2021/Volkskrant_2021.json')

df = df[df.cleantext.str.len()>5]
df = pd.DataFrame(df['cleantext'])

df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\&\w*;', '', x))
df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\n', '', x))
df['cleantext'] = df['cleantext'].map(lambda x: re.sub(r'\s\s+', ' ', x))
df['cleantext'] = df['cleantext'].map(lambda x: x.strip())

df = df[df.cleantext.str.len()>5]

In [13]:
df.head()

,cleantext
0,"Anja Hesselberth, leerkracht in groep 4 van OB..."
1,Want dat de Belastingdienst gediscrimineerd he...
2,"‘Eindelijk gerechtigheid’, zo reageerde Eric D..."
3,Beide wedstrijden waren fascinerend om te volg...
4,over de toeslagenaffaire en risicoprofilering ...


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-dutch")
tokenizer.pad_token = tokenizer.eos_token

from transformers import TextDataset

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,test_size=0.15)
print(train.shape)
print(test.shape)

(21448, 1)
(3785, 1)


In [16]:
!pip install datasets
from datasets import Dataset

dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [17]:
max_seq_length = 128
num_proc = 4

def tokenize_function(examples):
    # Remove empty lines
    examples["cleantext"] = [line for line in examples["cleantext"] if len(line) > 0 and not line.isspace()]
    return tokenizer(
        examples["cleantext"],
        truncation=True,
        max_length=max_seq_length,
    )

train_tokenized = dataset_train.map(
    tokenize_function,
    batched=True,
    num_proc=num_proc,
    remove_columns=["cleantext"],
)

test_tokenized = dataset_test.map(
    tokenize_function,
    batched=True,
    num_proc=num_proc,
    remove_columns=["cleantext"],
)

#0:   0%|          | 0/6 [00:00<?, ?ba/s]

#1:   0%|          | 0/6 [00:00<?, ?ba/s]

#2:   0%|          | 0/6 [00:00<?, ?ba/s]

#3:   0%|          | 0/6 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator =  DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("GroNLP/gpt2-small-dutch")


training_args = TrainingArguments(
    output_dir="drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21448
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2013


Step,Training Loss
500,4.254300
1000,3.981700
1500,3.897900
2000,3.844300


Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-800
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-800/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-800/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-1600
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-1600/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/checkpoint-1600/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2013, training_loss=3.9934079889865046, metrics={'train_runtime': 3941.6635, 'train_samples_per_second': 16.324, 'train_steps_per_second': 0.511, 'total_flos': 4203143626752000.0, 'train_loss': 3.9934079889865046, 'epoch': 3.0})

In [21]:
trainer.save_model()

Saving model checkpoint to drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant
Configuration saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/config.json
Model weights saved in drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant/pytorch_model.bin


# NRC

In [4]:
from transformers import pipeline

NRC = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-NRC', tokenizer="GroNLP/gpt2-small-dutch")

Downloading:   0%|          | 0.00/108 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [ ]:
NRC("Mark Rutte", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte (VVD) heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten. Dat schrijft hij na afloop van  aan demissionair minister Kajsa Ollongren (Binnenlandse Zaken, D66). Hij zei dat er �nog niets fout kan worden gemaakt� door toenmalig informateur Johan Remkes (, 17/2). De vraag is of VVD-minister Sigrid Kaag een verklaring wil afleggen tegen haar collega en partijgenoot Lilianne Ploumen (ChristenUnie'},
 {'generated_text': 'Mark Rutte (VVD) heeft in zijn persconferentie over het kabinetsberaad gezegd dat �de vraag is of er iets gebeurt� tijdens de ministerraad. Dat zei hij tegen demissionair minister van Defensie Ank Bijleveld (CDA), die donderdag naar Den Haag kwam om met informateur Mariëtte Hamer te werken. De afgelopen dagen hebben ook ministers en andere regeringsdeelnamesprocedures zich laten wachten, zo was volgens Van Dissel woensdag bekend gemaakt door.Van Dissel wilde geen commentaar geven'},
 {'generated_text': 'Mark R

In [ ]:
NRC("Sigrid Kaag", max_length=100, num_return_sequences=5)

[{'generated_text': 'Sigrid Kaag (VVD) is de nieuwe informateur voor het kabinet en moet nu worden opgevolgd door demissionair premier Mark Rutte (VVD). Dat heeft Ank Bijleveld (ChristenUnie) gezegd in. De VVD-leider had zich dinsdag gemeld als kandidate.Kaag, die van plan was om �de formatie aan te scherpen�, trad af na afloop van haar gesprek met Van der Staaij op een persconferentie nadat ze eerder deze week naar eigen zeggen '},
 {'generated_text': 'Sigrid Kaag (VVD) heeft de Kamer gevraagd om een onderzoek naar het Toeslagenaffaire te doen. Dat zegt ze tegen. �Ik hoop dat er duidelijkheid kan komen�, aldus Kaag na afloop van haar interview met informateur Herman Tjeenk Willink op NPO Radio 1. De Tweede Kamer kreeg afgelopen week al inzage in documenten waaruit blijkt dat D66-leider Sigrid Kaag en toenmalig VVD-voorman Mark Rutte jarenlang schuldig aan fraude zijn veroordeeld tot jarenlange celstraffen'},
 {'generated_text': 'Sigrid Kaag (D66) heeft in een brief geschreven dat er �

In [ ]:
NRC("Thierry Baudet", max_length=100, num_return_sequences=5)

[{'generated_text': 'Thierry Baudet, de oud-leider van Forum voor Democratie (FVD), is zondag door een rechter in Den Haag veroordeeld tot vier jaar celstraf. Dat maakte het Openbaar Ministerie donderdag bekend na berichtgeving. Het vonnis betekent dat Thierry Baudet en zijn advocaten niet langer kunnen aantonen dat zij strafbare feiten hebben begaan als er sprake was van �inmenging� of �extra mishandeling� bij hun cliënt.  De rechtbank oordeelde eerder dit jaar dat hij schuldig heeft gemaakt aan lidmaatschap van criminele organisaties'},
 {'generated_text': 'Thierry Baudet heeft vrijdag  een brief gestuurd aan de demissionair minister van Buitenlandse Zaken Hugo de Jonge (CDA) en staatssecretaris Mona Keijzer (Bakker). Daarin stellen zij dat het �steeds meer landen in Europa moeten gaan samenwerken�, zo meldt persbureau ANP. Volgens De Jonge is Nederland nog steeds niet bereid om samen te werken met andere EU-landen als er geen concrete afspraken worden gemaakt over migratie of klimaa

In [ ]:
NRC("Mark Rutte heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten.", max_length=100, num_return_sequences=5)

[{'generated_text': 'Mark Rutte heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten. Dat zei demissionair premier Mark Rutte (VVD) dinsdag tegen persbureau ANP, na berichtgeving van. �Ik hoop dat er meer openheid en transparantie komt�, reageerde hij op een  uit 2014 waarin bleek dat toenmalig Tweede Kamerlid Joost Eerdmans vertrouwelijke stukken had verstrekt aan minister De Jonge (Volksgezondheid, CDA). Volgens Rutte hebben alle betrokken partijen hem gevraagd om te zeggen hoe ze met elkaar omgegaan hadden'},
 {'generated_text': 'Mark Rutte heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten. �Het spijt me�, zei demissionair minister van Volksgezondheid Hugo de Jonge (CDA) na afloop van een interview op NPO Radio 2 met informateur Mariëtte Hamer: �Dit was niet mijn taak om dit te laten weten. Ik had geen idee wat ze zeiden en ik ben blij dat er nu aandacht is voor die foutjes.�Dat meldt persbureau ANP aan. Het'},
 {'

In [ ]:
NRC("Sigrid Kaag heeft in het debat over de Toeslagenaffaire excuses aangeboden voor haar eigen fouten.", max_length=100, num_return_sequences=5)

[{'generated_text': 'Sigrid Kaag heeft in het debat over de Toeslagenaffaire excuses aangeboden voor haar eigen fouten. Dat meldt persbureau ANP op basis van bronnen die anoniem hebben laten weten dat ze er �nog niet� aan doet om misstanden te voorkomen.Kaag zei vorige maand tijdens een Kamerdebat met D66-lijsttrekker Wopke Hoekstra (, 3/1). Ze gaf daarop toe: �Dit is iets heel persoonlijks, maar ik had geen idee waarom mensen zo boos zouden zijn als'},
 {'generated_text': 'Sigrid Kaag heeft in het debat over de Toeslagenaffaire excuses aangeboden voor haar eigen fouten. De PVV-leider zegt ze dat er �een schande van gemaakt wordt�, maar laat zich ook niet inhoudelijk commentaar geven op die opmerking.Kaag noemt zichzelf �de eerste vrouwelijke premier sinds 1945� en zei woensdagavond tijdens een tv-debat met oud-premier Dries van Agt (, 15/7). Ze zou hebben gezegd: �Het is tijd om te begrijpen waarom ik'},
 {'generated_text': 'Sigrid Kaag heeft in het debat over de Toeslagenaffaire excu

In [ ]:
NRC("Thierry Baudet heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten.", max_length=100, num_return_sequences=5)

[{'generated_text': 'Thierry Baudet heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten. �Ik kan me niet voorstellen dat er iets fout was gegaan�, schreef hij zondag op een online sociale medium. De voormalig PVV-lijsttrekker noemt zichzelf �zeer geschrokken� en vraagt zich af wat hem daarmee bedoelt. Hij zegt: �Het is erg moeilijk om iemand te veroordelen omdat je als volksvertegenwoordiger alleen nog maar verantwoordelijk bent geweest voor alle gevolgen van deze zaak.�'},
 {'generated_text': 'Thierry Baudet heeft in het debat over de Toeslagenaffaire excuses aangeboden voor zijn eigen fouten. �Ik ben oprecht blij dat ik er nu zo goed op heb gereageerd�, zegt hij na afloop van een gesprek met informateur Johan Remkes (, 8/8). De PVV en Forum voor Democratie (FVD) maakten zich maandag bekend om te laten weten dat ze hun steun niet meer zullen verlenen aan Van der Staaij: die partij zou hem nog steunen als verkenner-initiator'},
 {'generated_text': 'Thi

In [5]:
NRC("Thierry Baudet is voor", max_length=30, num_return_sequences=5)

[{'generated_text': 'Thierry Baudet is voor een groot deel van de coronapandemie �duidelijk� in het land. Dat blijkt uit '},
 {'generated_text': 'Thierry Baudet is voor het eerst op een podium in Amsterdam-Noord gestemd. De partij van Forum voor Democratie heeft zo�n 3,'},
 {'generated_text': 'Thierry Baudet is voor de tweede keer op rij in Amsterdam als lijsttrekker van Forum voor Democratie. Dat bevestigt zijn woordvoerder dinsdag na een live-uitzending'},
 {'generated_text': 'Thierry Baudet is voor een meerderheid van de Tweede Kamer  �niet bereid� om te regeren. Dat schrijft hij vrijdag op Twitter,'},
 {'generated_text': 'Thierry Baudet is voor de tweede keer in korte tijd een stemgerechtigde gast op televisie. De PVV-leider en oud-bestuursvoorzitter van'}]

In [8]:
NRC("Thierry Baudet is tegen", max_length=30, num_return_sequences=5)

[{'generated_text': 'Thierry Baudet is tegen het coronabeleid in Nederland. Dat heeft hij woensdag laten weten aan de Tweede Kamer tijdens een debat, meldt persbureau ANP'},
 {'generated_text': 'Thierry Baudet is tegen alle coronamaatregelen die in de Europese Unie komen. �Ik kan niet geloven dat er nu zoveel mogelijk mensen'},
 {'generated_text': 'Thierry Baudet is tegen de komst van een nieuwe regering in Nederland. Dat heeft zijn partij dinsdag  op basis van gesprekken met twee informateurs,'},
 {'generated_text': 'Thierry Baudet is tegen het einde van zijn politieke loopbaan �aan de weg�. Dat zei hij maandag tijdens een toespraak in Utrecht,'},
 {'generated_text': 'Thierry Baudet is tegen een einde te komen aan zijn campagne. Hij zegt dat hij niet langer de lijsttrekker moet worden van Forum voor Democratie, maar'}]

In [7]:
NRC("Sigrid Kaag is voor", max_length=30, num_return_sequences=5)

[{'generated_text': 'Sigrid Kaag is voor de derde keer minister van Economische Zaken in een kabinet. Dat blijkt uit haar  die maandag verschijnt, meldt persbureau ANP'},
 {'generated_text': 'Sigrid Kaag is voor de rechtbank in Den Haag al veroordeeld tot een celstraf van veertien maanden en tbs met dwangverpleging. De 39-'},
 {'generated_text': 'Sigrid Kaag is voor de derde keer in haar carrière minister van Buitenlandse Handel en Ontwikkelingssamenwerking. Dat maakte ze maandagavond bekend na afloop van een'},
 {'generated_text': 'Sigrid Kaag is voor de tweede keer in drie jaar tijd een minister geworden van Binnenlandse Zaken. Dat schrijft demissionair minister Hugo de Jonge'},
 {'generated_text': 'Sigrid Kaag is voor de rechter veroordeeld tot een gevangenisstraf van zes jaar, waarvan vier maanden voorwaardelijk. Dat bevestigt het Openbaar Ministerie (OM)'}]

In [9]:
NRC("Sigrid Kaag is tegen", max_length=30, num_return_sequences=5)

[{'generated_text': 'Sigrid Kaag is tegen het afschaffen van de avondklok. Dat zei demissionair premier Mark Rutte donderdag op een persconferentie, meldt persbureau ANP'},
 {'generated_text': 'Sigrid Kaag is tegen de coronamaatregelen in Nederland. Volgens haar gaat het �onvoldoende om ons te beschermen�'},
 {'generated_text': 'Sigrid Kaag is tegen het nieuwe coronamaatregelen. Dat heeft ze vrijdag in een brief aan minister Hugo de Jonge (Volksgezondheid,'},
 {'generated_text': 'Sigrid Kaag is tegen de coronapandemie. Dat heeft ze maandagavond gezegd tijdens een persconferentie bij het Rijksinstituut voor Volksgezondheid en Milieu'},
 {'generated_text': 'Sigrid Kaag is tegen het coronabeleid in de Kamer. De demissionair minister van Volksgezondheid (Volksgezondheid, Welzijn en Sport'}]

In [6]:
NRC("Mark Rutte is voor", max_length=30, num_return_sequences=5)

[{'generated_text': 'Mark Rutte is voor het eerst sinds zijn aantreden premier geworden als minister van Buitenlandse Zaken. Hij volgt Mark Rutte (VVD) in de Tweede Kamer, die'},
 {'generated_text': 'Mark Rutte is voor het eerst sinds 1 juli premier geworden. Dat heeft de demissionair minister van Economische Zaken en Klimaat, Eric Wiebes (Economische Zaken'},
 {'generated_text': 'Mark Rutte is voor het eerst sinds zijn eerste kabinet-Rutte II niet meer in de Tweede Kamer. Dat heeft demissionair minister Hugo de Jonge'},
 {'generated_text': 'Mark Rutte is voor het eerst op rij de premier van Nederland. Dat heeft demissionair minister Ferd Grapperhaus (CDA) dinsdag bekendgemaakt, melden  en'},
 {'generated_text': 'Mark Rutte is voor het eerst in zijn regeerperiode op dit moment leider van de oppositie. In een interview met  zegt premier Mark Rutte dat hij �'}]

In [10]:
NRC("Mark Rutte is tegen", max_length=30, num_return_sequences=5)

[{'generated_text': 'Mark Rutte is tegen de invoering van een Toeslagenaffaire. Dat schrijft hij maandag in  aan premier Mark Rutte en Tweede Kamerlid Pieter Omtzigt,'},
 {'generated_text': 'Mark Rutte is tegen de coronamaatregelen van het RIVM. Dat meldt persbureau ANP op basis van een gesprek tussen informateur Mariëtte Hamer'},
 {'generated_text': 'Mark Rutte is tegen het nieuwe coronamaatregelen, die volgens hem een �ernstig probleem� zijn geworden. Dat zegt demissionair'},
 {'generated_text': 'Mark Rutte is tegen de coronaregels. Dat zei hij dinsdagavond in het NOS-toespraak tijdens een gesprek met informateur Herman Tjeenk Will'},
 {'generated_text': 'Mark Rutte is tegen versoepelingen in de coronaregels. Hij zei vorige week dat hij zich niet wil laten wachten op �de laatste fase�'}]

#Volkskrant

In [ ]:
from transformers import pipeline

Volkskrant = pipeline('text-generation',model='drive/MyDrive/CS/Thesis/GPT2/gpt2-Volkskrant', tokenizer="GroNLP/gpt2-small-dutch")